# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#500 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846800922585                   -0.70    4.0
  2   -7.852310518702       -2.26       -1.53    1.0
  3   -7.852646102547       -3.47       -2.52    3.2
  4   -7.852646677861       -6.24       -3.35    2.2
  5   -7.852646685882       -8.10       -4.78    1.0
  6   -7.852646686725       -9.07       -5.18    4.2
  7   -7.852646686730      -11.34       -5.92    2.0
  8   -7.852646686730      -12.53       -7.37    2.0
  9   -7.852646686730      -14.45       -7.66    2.2
 10   -7.852646686730   +  -14.57       -9.27    1.5
 11   -7.852646686730      -14.35       -9.66    3.2
 12   -7.852646686730   +  -14.45       -8.47    2.2
 13   -7.852646686730   +  -14.75       -8.63    2.0
 14   -7.852646686730      -14.45       -8.15    2.5
 15   -7.852646686730   +  -15.05       -8.79    2.8
 16   -7.852646686730   +  -15.05       -9.14    1.0
 17   -7.852646686730   +    -Inf       -9.19 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846763745807                   -0.70           4.2
  2   -7.852527879323       -2.24       -1.62   0.80    5.0
  3   -7.852609033422       -4.09       -2.79   0.80    5.2
  4   -7.852646564543       -4.43       -3.55   0.80    7.2
  5   -7.852646683472       -6.92       -4.48   0.80    9.2
  6   -7.852646686647       -8.50       -5.06   0.80   10.5
  7   -7.852646686724      -10.11       -6.22   0.80   11.8
  8   -7.852646686730      -11.27       -7.27   0.80   13.5
  9   -7.852646686730      -13.60       -7.65   0.80   16.0
 10   -7.852646686730      -14.57       -9.34   0.80   15.8
 11   -7.852646686730   +  -14.57       -9.30   0.80   18.5
 12   -7.852646686730   +    -Inf      -10.49   0.80   18.8
 13   -7.852646686730      -14.75      -11.57   0.80   20.8
 14   -7.852646686730   +  -14.75      -11.97   0.80   23.0
 15   -7.852646686730      -15.05      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.422846e+01     3.186559e+00
 * time: 0.5332310199737549
     1     8.736661e-01     1.588767e+00
 * time: 0.852647066116333
     2    -1.702622e+00     1.802809e+00
 * time: 0.8812010288238525
     3    -3.857434e+00     1.569343e+00
 * time: 0.923332929611206
     4    -5.029122e+00     1.390604e+00
 * time: 0.9650599956512451
     5    -6.773738e+00     1.003687e+00
 * time: 1.0057580471038818
     6    -7.441644e+00     3.822100e-01
 * time: 1.0470049381256104
     7    -7.698827e+00     2.405746e-01
 * time: 1.0764310359954834
     8    -7.780476e+00     9.623952e-02
 * time: 1.1146290302276611
     9    -7.822293e+00     9.355485e-02
 * time: 1.1436290740966797
    10    -7.835727e+00     5.022529e-02
 * time: 1.1728110313415527
    11    -7.842691e+00     4.910242e-02
 * time: 1.2027029991149902
    12    -7.845360e+00     4.639681e-02
 * time: 1.2314949035644531
    13    -7.847363e+00     3.672318e-02
 * time: 1.261100053787

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846856807076                   -0.70    4.2
  2   -7.852325259537       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686705                   -2.54
  2   -7.852646686730      -10.61       -5.89
  3   -7.852646686730   +    -Inf      -12.55
|ρ_newton - ρ_scf|  = 8.924497032605167e-13
|ρ_newton - ρ_scfv| = 1.3753550859121267e-14
|ρ_newton - ρ_dm|   = 5.984292408171641e-10
